In [736]:
from ipynb.fs.full.Mimic import *
from scipy.stats import norm
from sklearn.neighbors import NearestNeighbors
import pickle
import numpy as np

In [733]:
def fill(points_x, points_y, data):
    gen, prm = scaled_norm().fit(points_x, points_y, data) # Gets the generated points and the fitted parameters with the given histogram
    num = np.maximum(0, gen - points_y).round().astype(int) # The number of points added by IMITATE
    # The next 2 lines translate the fitted histogram into actual points by sampling the desired number of points uniformly within each bin
    step = points_x[1] - points_x[0]
    gen = np.concatenate([np.random.uniform(points_x[i]-step/2,points_x[i]+step/2,num[i]) for i in range(len(points_x))])
    return prm, gen

def param0(B, bins):
    step = (B.max()-B.min())/bins
    x = np.linspace(B.min(), B.max(), bins+1)[:-1] + step/2 # The x values for each bin in the histogram
    r = np.bincount((((B -B.min()) /(B.max()-B.min())) * bins - 1e-6).astype(int)) # The height of each bin in the histogram
    prm, gen = fill(x, r, B) # Getting the points generated by IMITATE
    if len(gen) <= 10: # With fewer than 10 points, the confidence measure is undefined, so zero is returned
        return prm, 0
    nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(gen.reshape(-1, 1)) # Getting the 10 nearest neighbours
    distances, _ = nbrs.kneighbors(gen.reshape(-1, 1))
    return prm, 1/distances.mean() 

In [734]:
# iterations = 1000
# res = np.zeros((iterations, 5))
# min_b = 5 # The minimum number of bins considered
# max_b = 100 # The maximum number of bins considered
# ran = np.arange(min_b, max_b + 1)

# for i in range(iterations):
#     thres = np.random.random() # A randomised threshold for the artificial selection bias
#     alpha = np.random.random() # A randomised harshness fot the artificial selection bias
#     size = int(np.random.random()*45000+5000) # A randomised sample size
#     thres = norm.ppf(thres) # Converting the threshold from a quantile to a point on the Gaussian

#     D = np.random.normal(0, 1, size = size) # Sampling the unbiased dataset
#     B = D[np.random.random(size = size) < alpha + (1-alpha) * (D < thres)] # Applying the artificial selection bias
#     n = B.shape[0]

#     kl = np.zeros(max_b - min_b + 1) # The KL-divergence for each possible number of bins
#     conf = np.zeros(max_b - min_b + 1) # The confidence score for each possible number of bins
#     for bins in ran:
#         aaa, c = param0(B, bins)
#         conf[bins - min_b] = c
#         _, mu, sigma = aaa
#         kl[bins - min_b] = np.log(sigma)+(1+mu**2)/(2*sigma**2)-0.5 # See equation 4.20

#     log_l = np.zeros(max_b - min_b + 1)
#     for bins in ran:
#         inds = np.floor((B - B.min()) / (B.ptp() + 1e-12) * bins).astype(int) # The bin indices for each data point
#         pdf = np.bincount(inds) / n * bins / B.ptp() # Evaluating the pdf (where the histogram is the model)
#         log_l[bins - min_b] = np.log(pdf[inds]).sum() # Adding up the log-likelihood corresponding to the number of bins

#     aic = -2 * log_l + 2 * ran
#     aicc = aic + (2 * np.square(ran) + 2 * ran) / (n - ran - 1)
#     bic = -2 * log_l + np.log(n) * ran
#     hqc = -2 * log_l + 2 * ran * np.log(np.log(n))

#     og_kl = kl[conf.argmax()] # The KL-divergence associated with the original approach is the one corresponding to the largest confidence score
#     aic_kl = kl[aic.argmin()] # The KL-divergence associated with AIC is the one corresponding to the smallest AIC score
#     aicc_kl = kl[aicc.argmin()] # The KL-divergence associated with AICc is the one corresponding to the smallest AICc score
#     bic_kl = kl[bic.argmin()] # The KL-divergence associated with BIC is the one corresponding to the smallest BIC score
#     hqc_kl = kl[hqc.argmin()] # The KL-divergence associated with HQC is the one corresponding to the smallest HQC score

#     res[i] = [og_kl, aic_kl, aicc_kl, bic_kl, hqc_kl]
#     print(i, end = ' ')

# res = np.where(np.repeat((res.sum(1)>20)[:,None], 5, 1), res/res.sum(1)[:,None]*20, res)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [745]:
# with open('Results/number_of_bins.pickle', 'wb') as f:
#     pickle.dump(res, f, pickle.HIGHEST_PROTOCOL)